In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
from tqdm import tqdm
import pickle

df = pd.read_csv("../data/data_raw.csv")
df = df.drop('id', 1)

y = df['diagnosis']
X = df.loc[:, df.columns != 'diagnosis']

In [27]:
final_data = []
final_models = []
maj_class = []
final_train = []

for i in tqdm(range(10,110)):
    
    rs = 42*i
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=rs)
    
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)
    
    std_ftrs = list(X.columns)
    preprocessor = ColumnTransformer(transformers=[('std', StandardScaler(), std_ftrs)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor)])
    
    X_train = pipe.fit_transform(X_train)
    X_test = pipe.transform(X_test)
    
    clf = LogisticRegression(penalty='l2', solver='saga', C=1, random_state=rs, max_iter=100000)
    
    clf.fit(X_train, y_train)
    
    final_data.append((X_test, y_test))
    final_train.append((X_train, y_train))
    
    final_models.append(clf)
    maj_class.append(max(set(y_train), key=list(y_train).count))
    

100%|██████████| 100/100 [00:09<00:00, 10.33it/s]


In [28]:
file = open('../results/final_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

file = open('../data/final_test_data.save', 'wb')
pickle.dump(final_data, file)
file.close()

file = open('../data/final_train_data.save', 'wb')
pickle.dump(final_train, file)
file.close()



In [26]:
from statistics import mean
mean(maj_class)

0